In [1]:
import torch
from torch import nn
import bert
import tensorflow as tf
from typing import Optional
from einops import rearrange, repeat

2024-01-21 11:20:22.338908: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-21 11:20:22.939708: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-21 11:20:24.236586: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# Credit to microsoft phi-2 code (put something like this in final piece of code)

In [ ]:
# testing Embedding dimensions and calculations (done)
class Embedding(nn.Module):
    """Token embedding with dropout."""
    def __init__(self, vocab_size, n_embd) -> None:
        super().__init__()
        self.wte = nn.Embedding(vocab_size, n_embd)

    def forward(self, input_ids: torch.LongTensor) -> torch.FloatTensor:
        input_shape = input_ids.size()
        input_ids = input_ids.view(-1, input_shape[-1])
        hidden_states = self.wte(input_ids)
        return hidden_states

# testing py
py_layer = Embedding(vocab_size=3000, n_embd=768)
input = torch.randint(0, 3000, (4,))
# print(py_layer(input))
# testing tf
matrix = py_layer.wte._parameters['weight'].data.numpy()
matrix = tf.convert_to_tensor(matrix)
tf_layer = bert.Embedding(4, 768, matrix)
# print(tf_layer(input))

In [ ]:
# testing _apply_rotary_emb() (done)
def MY_apply_rotary_emb(x, cos, sin): # code mostly taken from original
    assert(len(x.shape) == 4)
    assert(len(cos.shape) == 2)
    _, seqlen, _, _ = x.shape
    _, rotary_dim = cos.shape
    rotary_dim *= 2
    x_rot = x[:, :, :, :rotary_dim]
    # print("x_rot", x_rot.shape)
    x_pass = x[:, :, :, rotary_dim:]
    x1, x2 = tf.split(x_rot, 2, axis=-1) # assumption that tf.split behaves the same as nn.chunk
    c = tf.expand_dims(cos[:seqlen], axis=1)
    s = tf.expand_dims(sin[:seqlen], axis=1)
    x1, x2, c, s = [tf.cast(t, tf.float32) for t in [x1, x2, c, s]]
    for i in [x1, x2, c, s]: print(i.shape)
    x_rot = tf.concat([x1 * c - x2 * s, x1 * s + x2 * c], axis=-1)
    x_rot = tf.cast(x_rot, x.dtype) # might fail due to passing in x.dtype
    return tf.concat([x_rot, x_pass], axis=-1)


def _apply_rotary_emb(
    x: torch.FloatTensor,
    cos: torch.FloatTensor,
    sin: torch.FloatTensor,
) -> torch.FloatTensor:
    # print("x.shape:", x.shape)
    _, seqlen, _, _ = x.shape
    _, rotary_dim = cos.shape
    rotary_dim *= 2
    x_rot = x[:, :, :, :rotary_dim]
    # print("x_rot", x_rot.shape)
    x_pass = x[:, :, :, rotary_dim:]
    x1, x2 = x_rot.chunk(2, dim=-1)
    c, s = rearrange(cos[:seqlen], "s d -> s 1 d"), rearrange(sin[:seqlen], "s d -> s 1 d")
    x1, x2, c, s = [t.to(dtype=torch.float32) for t in [x1, x2, c, s]]
    for i in [x1, x2, c, s]: print(i.shape)
    x_rot = torch.cat([x1 * c - x2 * s, x1 * s + x2 * c], axis=-1).to(x.dtype)
    return torch.cat([x_rot, x_pass], axis=-1)

last_dim = 5 # it seems this is neccesary
x = torch.randn([2,3,2,last_dim*2]) 
cos = torch.randn([4,last_dim])
sin = torch.randn([4,last_dim])
# print(_apply_rotary_emb(x, cos, sin))
# print("-----")
x = tf.constant(x.numpy())
cos = tf.constant(cos.numpy())
sin = tf.constant(sin.numpy())
# print(MY_apply_rotary_emb(x, cos, sin))

In [ ]:
# testing _apply_rotary_emb_kv (done)
def MY_apply_rotary_emb_kv(kv, cos, sin, cos_k=None, sin_k=None):
    assert(len(kv.shape) == 5)
    assert(len(cos.shape) == 2)
    if cos_k is not None: assert(len(cos_k.shape) == 4)
    if cos_k is not None: assert(cos_k.shape == sin_k.shape)
    _, seqlen, _, _, _ = kv.shape
    _, rotary_dim = cos.shape
    rotary_dim *= 2
    k_rot = kv[:, :, 0, :, :rotary_dim]
    k_pass = kv[:, :, 0, :, rotary_dim:]
    k1, k2 = tf.split(k_rot, 2, axis=-1)
    c = tf.expand_dims(cos[:seqlen], axis=1)
    s = tf.expand_dims(sin[:seqlen], axis=1)
    k1, k2, c, s = [tf.cast(t, tf.float32) for t in [k1, k2, c, s]]
    k_rot = tf.concat([k1 * c - k2 * s, k1 * s + k2 * c], axis=-1)
    k_rot = tf.cast(k_rot, kv.dtype)
    return tf.concat([
        tf.expand_dims(tf.concat([k_rot, k_pass], axis=-1), axis=2),
        kv[:, :, 1:2, :, :],
    ], axis=2)

def _apply_rotary_emb_kv(
    kv: torch.FloatTensor,
    cos: torch.FloatTensor,
    sin: torch.FloatTensor,
    cos_k= None,
    sin_k= None,
) -> torch.FloatTensor:
    _, seqlen, _, _, _ = kv.shape
    _, rotary_dim = cos.shape
    rotary_dim *= 2
    k_rot = kv[:, :, 0, :, :rotary_dim]
    k_pass = kv[:, :, 0, :, rotary_dim:]
    k1, k2 = k_rot.chunk(2, dim=-1)
    c, s = rearrange(cos[:seqlen], "s d -> s 1 d"), rearrange(sin[:seqlen], "s d -> s 1 d")
    k1, k2, c, s = [t.to(dtype=torch.float32) for t in [k1, k2, c, s]]
    k_rot = torch.cat([k1 * c - k2 * s, k1 * s + k2 * c], axis=-1).to(kv.dtype)
    return torch.cat(
        [
            torch.cat([k_rot, k_pass], axis=-1).unsqueeze(2),
            kv[:, :, 1:2, :, :],
        ],
        axis=2,
    )

last_dim = 5 
kv = torch.randn([2,3,2, 8, last_dim*2]) 
cos = torch.randn([4,last_dim])
sin = torch.randn([4,last_dim])
cos_k = torch.randn([4, 5, 6, last_dim])
sin_k = torch.randn([4, 5, 6, last_dim])
# print(_apply_rotary_emb_kv(kv, cos, sin, cos_k, sin_k))
# print("-----")
kv = tf.constant(kv.numpy())
cos = tf.constant(cos.numpy())
sin = tf.constant(sin.numpy())
cos_k = tf.constant(cos_k.numpy())
sin_k = tf.constant(sin_k.numpy())

# print(MY_apply_rotary_emb_kv(kv, cos, sin, cos_k, sin_k))

In [ ]:
# testing _apply_rotary_emb_qkv (done)

def MY_apply_rotary_emb_qkv(qkv, cos, sin, cos_k=None, sin_k=None):
    assert(len(qkv.shape) == 5)
    assert(len(cos.shape) == 2)
    if cos_k is not None: assert(len(cos_k.shape) == 4)
    if cos_k is not None: assert(cos_k.shape == sin_k.shape)
    _, seqlen, _, _, _ = qkv.shape
    _, rotary_dim = cos.shape
    rotary_dim *= 2
    q_rot = qkv[:, :, 0, :, :rotary_dim]
    q_pass = qkv[:, :, 0, :, rotary_dim:]
    k_rot = qkv[:, :, 1, :, :rotary_dim]
    k_pass = qkv[:, :, 1, :, rotary_dim:]
    q1, q2 = tf.split(q_rot, 2, axis=-1)
    k1, k2 = tf.split(k_rot, 2, axis=-1)
    c = tf.expand_dims(cos[:seqlen], axis=1)
    s = tf.expand_dims(sin[:seqlen], axis=1)
    q1, q2, k1, k2, c, s = [tf.cast(t, tf.float32) for t in [q1, q2, k1, k2, c, s]]
    q_rot = tf.concat([q1 * c - q2 * s, q1 * s + q2 * c], axis=-1)
    q_rot = tf.cast(q_rot, qkv.dtype)
    k_rot = tf.concat([k1 * c - k2 * s, k1 * s + k2 * c], axis=-1)
    k_rot = tf.cast(k_rot, qkv.dtype)
    return tf.concat([
        tf.expand_dims(tf.concat([q_rot, q_pass], axis=-1), axis=2),
        tf.expand_dims(tf.concat([k_rot, k_pass], axis=-1), axis=2),
        qkv[:, :, 2:3, :, :],
    ], axis=2)

def _apply_rotary_emb_qkv(
    qkv: torch.FloatTensor,
    cos: torch.FloatTensor,
    sin: torch.FloatTensor,
    cos_k=None,
    sin_k=None,
) -> torch.FloatTensor:
    _, seqlen, _, _, _ = qkv.shape
    _, rotary_dim = cos.shape
    rotary_dim *= 2
    q_rot = qkv[:, :, 0, :, :rotary_dim]
    q_pass = qkv[:, :, 0, :, rotary_dim:]
    k_rot = qkv[:, :, 1, :, :rotary_dim]
    k_pass = qkv[:, :, 1, :, rotary_dim:]
    q1, q2 = q_rot.chunk(2, dim=-1)
    k1, k2 = k_rot.chunk(2, dim=-1)
    c, s = rearrange(cos[:seqlen], "s d -> s 1 d"), rearrange(sin[:seqlen], "s d -> s 1 d")
    q1, q2, k1, k2, c, s = [t.to(dtype=torch.float32) for t in [q1, q2, k1, k2, c, s]]
    q_rot = torch.cat([q1 * c - q2 * s, q1 * s + q2 * c], axis=-1).to(qkv.dtype)
    k_rot = torch.cat([k1 * c - k2 * s, k1 * s + k2 * c], axis=-1).to(qkv.dtype)
    return torch.cat(
        [   torch.cat([q_rot, q_pass], axis=-1).unsqueeze(2),
            torch.cat([k_rot, k_pass], axis=-1).unsqueeze(2),
            qkv[:, :, 2:3, :, :],
        ],
        axis=2,
    )

last_dim = 5 
qkv = torch.randn([2,3,2, 8, last_dim*2])
cos = torch.randn([4,last_dim])
sin = torch.randn([4,last_dim])
cos_k = torch.randn([4, 5, 6, last_dim])
sin_k = torch.randn([4, 5, 6, last_dim])
# print(_apply_rotary_emb_kv(qkv, cos, sin, cos_k, sin_k))
# print("-----")
qkv = tf.constant(qkv.numpy())
cos = tf.constant(cos.numpy())
sin = tf.constant(sin.numpy())
cos_k = tf.constant(cos_k.numpy())
sin_k = tf.constant(sin_k.numpy())
# print(MY_apply_rotary_emb_kv(qkv, cos, sin, cos_k, sin_k))

In [ ]:
# testing rotary embedding layer (done)
class MyRotaryEmbedding(tf.Module):
    def __init__(
        self, 
        dim: int,
        base: int = 10000,
        scale_base: float = None,
        pos_idx_in_fp32: bool = True,
        max_position_embeddings: int = 2048,
        name=None,
        **kwargs
        ):
        super().__init__(name)
        self.dim = dim
        self.base = float(base)
        self.scale_base = scale_base
        self.pos_idx_in_fp32 = pos_idx_in_fp32
        self.max_position_embeddings = max_position_embeddings
        # Generate and save the inverse frequency buffer (non-trainable)
        self.inv_freq = self._compute_inv_freq()
        # Generate and save the scale buffer (non-trainable)
        self.scale = (
            (tf.range(0, dim, 2, dtype=tf.float32) + 0.4 * dim) / (1.4 * dim)
            if scale_base is not None
            else None)
        self._update_cos_sin_cache(max_position_embeddings, dtype=tf.float32)

    def _compute_inv_freq(self):
        return 1.0 / (self.base ** (tf.range(0, self.dim, 2, dtype=tf.float32) / self.dim))

    def _update_cos_sin_cache(self, seqlen:int, dtype):
        self._seq_len_cached = seqlen
        # fp32 is preferred since the output of `torch.arange` can be quite large
        # and bf16 would lose a lot of precision
        if self.pos_idx_in_fp32:
            t = tf.range(seqlen, dtype=tf.float32)
            if self.inv_freq.dtype != tf.float32:
                inv_freq = self._compute_inv_freq()
            else:
                inv_freq = self.inv_freq
        else:
            t = tf.range(seqlen, dtype=self.inv_freq.dtype)
            inv_freq = self.inv_freq
        # `torch.outer` is preferred since `torch.einsum` converts from fp32 to fp16 if used with AMP
        # tensorflow does not appear to do what the top comment states
        freqs = tf.einsum('i,j->ij', t, inv_freq)
        if self.scale is None:
            self._cos_cached = tf.cast(tf.cos(freqs), dtype=dtype)
            self._sin_cached = tf.cast(tf.sin(freqs), dtype=dtype)
        else:
            power = (tf.range(seqlen, dtype=self.scale.dtype) - seqlen // 2) / self.scale_base
            scale = self.scale ** tf.expand_dims(power, axis=1)
            # Force the scale multiplciation to happen in fp32
            self._cos_cached = tf.cast((tf.cos(freqs) * scale), dtype=dtype)
            self._sin_cached = tf.cast((tf.sin(freqs) * scale), dtype=dtype)
            self._cos_k_cached = tf.cast((tf.cos(freqs) / scale), dtype=dtype)
            self._sin_k_cached = tf.cast((tf.sin(freqs) / scale), dtype=dtype)
    
    def __call__(self, qkv, kv, seqlen_offset, **kwargs):
        if (
            self._seq_len_cached < qkv.shape[1] + seqlen_offset
            or self._cos_cached.device != qkv.device
            or self._cos_cached.dtype != qkv.dtype
            # or (self.training and self._cos_cached.is_inference()) # look into this
        ):
            self._update_cos_sin_cache(qkv.shape[1] + seqlen_offset, dtype=qkv.dtype)
            print("case 1")
        else:
            print("case 2")
        if kv is None:
            return MY_apply_rotary_emb_qkv(
                qkv,
                self._cos_cached[seqlen_offset:],
                self._sin_cached[seqlen_offset:],)
        else:
            # print("qkv:", qkv.shape)
            # print("_cos_cached:", self._cos_cached[seqlen_offset:].shape)
            # print("_sin_cached:", self._sin_cached[seqlen_offset:].shape)
            q = MY_apply_rotary_emb(
                qkv,
                self._cos_cached[seqlen_offset:],
                self._sin_cached[seqlen_offset:],)
            kv = MY_apply_rotary_emb_kv(
                kv,
                self._cos_cached[seqlen_offset:],
                self._sin_cached[seqlen_offset:],)
            return q, kv
    ################################
class RotaryEmbedding(nn.Module):
    """Rotary positional embedding (RoPE).
    Reference:
        RoFormer: Enhanced Transformer with Rotary Position Embedding.
        https://arxiv.org/pdf/2104.09864.pdf.
    """

    def __init__(
        self,
        dim: int,
        base: int = 10000,
        scale_base= None,
        pos_idx_in_fp32: bool = True,
        max_position_embeddings: int = 2048,
        device = None,
        **kwargs,
    ) -> None:
        super().__init__()
        if scale_base is not None:
            raise NotImplementedError
        self.dim = dim
        self.base = float(base)
        self.scale_base = scale_base
        self.pos_idx_in_fp32 = pos_idx_in_fp32
        self.max_position_embeddings = max_position_embeddings
        self.device = device
        # Generate and save the inverse frequency buffer (non-trainable)
        inv_freq = self._compute_inv_freq(device)
        self.register_buffer("inv_freq", inv_freq, persistent=False)

        # Generate and save the scale buffer (non-trainable)
        scale = (
            (torch.arange(0, dim, 2, device=device, dtype=torch.float32) + 0.4 * dim) / (1.4 * dim)
            if scale_base is not None
            else None
        )
        self.register_buffer("scale", scale, persistent=False)

        # Initialize cached attributes since ONNX can't rely on dynamic initialization
        self._update_cos_sin_cache(max_position_embeddings, device=device, dtype=torch.float32)

    def _compute_inv_freq(self, device: Optional[str] = None) -> torch.FloatTensor:
        return 1.0 / (self.base ** (torch.arange(0, self.dim, 2, device=device, dtype=torch.float32) / self.dim))

    def _update_cos_sin_cache(
        self,
        seqlen: int,
        device: Optional[str] = None,
        dtype: Optional[torch.dtype] = None,
    ) -> None:
        self._seq_len_cached = seqlen

        # fp32 is preferred since the output of `torch.arange` can be quite large
        # and bf16 would lose a lot of precision
        if self.pos_idx_in_fp32:
            t = torch.arange(seqlen, device=device, dtype=torch.float32)
            if self.inv_freq.dtype != torch.float32:
                inv_freq = self._compute_inv_freq(device=device)
            else:
                inv_freq = self.inv_freq
        else:
            t = torch.arange(seqlen, device=device, dtype=self.inv_freq.dtype)
            inv_freq = self.inv_freq

        # `torch.outer` is preferred since `torch.einsum` converts from fp32 to fp16 if used with AMP
        freqs = torch.outer(t, inv_freq)
        if self.scale is None:
            self._cos_cached = torch.cos(freqs).to(dtype)
            self._sin_cached = torch.sin(freqs).to(dtype)
        else:
            power = (
                torch.arange(seqlen, dtype=self.scale.dtype, device=self.scale.device) - seqlen // 2
            ) / self.scale_base
            scale = self.scale.to(device=power.device) ** rearrange(power, "s -> s 1")

            # Force the scale multiplication to happen in fp32
            self._cos_cached = (torch.cos(freqs) * scale).to(dtype)
            self._sin_cached = (torch.sin(freqs) * scale).to(dtype)
            self._cos_k_cached = (torch.cos(freqs) / scale).to(dtype)
            self._sin_k_cached = (torch.sin(freqs) / scale).to(dtype)

    def forward(
        self,
        qkv: torch.Tensor,
        kv: Optional[torch.Tensor] = None,
        seqlen_offset: int = 0,
        **kwargs,
    ):
        if (
            self._seq_len_cached < qkv.shape[1] + seqlen_offset
            or self._cos_cached.device != qkv.device
            or self._cos_cached.dtype != qkv.dtype
            or (self.training and self._cos_cached.is_inference())
        ):
            self._update_cos_sin_cache(qkv.shape[1] + seqlen_offset, device=qkv.device, dtype=qkv.dtype)
            print("case 1")
        else:
            print("case 2")
        if kv is None:
            return _apply_rotary_emb_qkv(
                qkv,
                self._cos_cached[seqlen_offset:],
                self._sin_cached[seqlen_offset:],
            )
        else:
            # print("qkv:", qkv.shape)
            # print("_cos_cached:", self._cos_cached[seqlen_offset:].shape)
            # print("_sin_cached:", self._sin_cached[seqlen_offset:].shape)
            q = _apply_rotary_emb(
                qkv,
                self._cos_cached[seqlen_offset:],
                self._sin_cached[seqlen_offset:],
            )
            kv = _apply_rotary_emb_kv(
                kv,
                self._cos_cached[seqlen_offset:],
                self._sin_cached[seqlen_offset:],
            )

            return q, kv


dim = 5
base = 5000
scale_base = None
pos_idx_in_fp32 = False
max_position_embeddings = 2048
device = 'cpu'

rot_emb = RotaryEmbedding(dim, base, scale_base, pos_idx_in_fp32, max_position_embeddings, device)
tf_rot_emb = MyRotaryEmbedding(dim, base, scale_base, pos_idx_in_fp32, max_position_embeddings)

# test with kv being something, and being None

# it appears that if kv will be none, then qkv must be 4d rather than 5d

last_dim = 5
# qkv = torch.randn([2,3,2, 8, last_dim*2]) 
qkv = torch.randn([2,3, 8, last_dim*2]) # making qkv
kv = torch.randn([2,3,2, 8, last_dim*2])
seqlen_offset = 4
print(rot_emb(qkv, kv, seqlen_offset))
print("-----")
qkv = tf.constant(qkv.numpy())
kv = tf.constant(kv.numpy())
print(tf_rot_emb(qkv, kv, seqlen_offset))

In [ ]:
# testing Linear Layer
layer = torch.nn.Linear(3, 5)
input = torch.randn([7,3])
print(layer(input))
weights = tf.cast(torch.detach(layer.weight).numpy(), dtype=tf.float32)
bias = tf.cast(torch.detach(layer.bias).numpy(), dtype=tf.float32)
linear = bert.Dense(3, 5, weights, is_biased=True, bias=bias)
input = tf.cast(input.numpy(), dtype=tf.float32)
print(linear(input))

In [3]:
import math
class NewGELU(tf.Module):
    def __init__(self, name=None):
        super().__init__(name)
    def __call__(self, input):
        return 0.5 * input * (1.0 + tf.tanh(math.sqrt(2.0 / math.pi) * (input + 0.044715 * tf.pow(input, 3.0))))

In [5]:
# MLP (need to fill in the weight, bias, and activation) (works)

class MyMLP(tf.Module):
    def __init__(self, n_inner:int, n_embd:int, name=None): # manually written in act-fn()
        super().__init__(name)
        self.fc1 = bert.Dense(n_embd, n_inner, weights=None, is_biased=True, bias=None)
        self.fc2 = bert.Dense(n_inner, n_embd, weights=None, is_biased=True, bias=None)
        self.act = NewGELU
    def __call__(self, hidden_states):
        hidden_states = self.fc1(hidden_states)
        hidden_states = self.act(hidden_states)
        hidden_states = self.fc2(hidden_states)
        return hidden_states

class NewGELUActivation(nn.Module): # grabbed from hugging face
    def forward(self, input):
        return 0.5 * input * (1.0 + torch.tanh(math.sqrt(2.0 / math.pi) * (input + 0.044715 * torch.pow(input, 3.0))))
    

# input = torch.randn([2,3])
# print(input)
# layer = NewGELUActivation()
# print(layer(input))
# input = tf.constant(input.numpy())
# layer = NewGELU()
# print(layer(input))

tensor([[-1.6362,  0.2685, -0.0438],
        [ 1.4699, -2.7153, -1.5775]])
tensor([[-0.0834,  0.1627, -0.0211],
        [ 1.3656, -0.0085, -0.0907]])
tf.Tensor(
[[-0.08344983  0.16265765 -0.02111891]
 [ 1.3656254  -0.00851566 -0.09065148]], shape=(2, 3), dtype=float32)


In [15]:
# the non-masking parts look correct

# note: I dont know the notation of the bool mask (does "true" mean to mask or not)
# what is probably true: 1 (or true) means to keep

# assumption that must be checked: assume that True means to replace --> *** FIX THIS
def mask_fill(matrix, mask, value):
    negmask = 1 - mask
    return (matrix * mask) + (negmask * value)

class MySelfAttention(tf.Module):
    def __init__(self, softmax_scale:float, name=None):
        super().__init__(name)
        self.softmax_scale = softmax_scale
    def __call__(self, qkv, key_padding_mask=None):
        batch_size, seqlen = qkv.shape[0], qkv.shape[1]
        q, k, v = tf.unstack(qkv, axis=2)
        q = tf.cast(q, dtype=tf.float32)
        k = tf.cast(k, dtype=tf.float32)
        softmax_scale = self.softmax_scale
        # Autocast is manually disabled to avoid `torch.einsum` performing the operation
        scores = tf.einsum("bthd,bshd->bhts", q, k * softmax_scale)
        if key_padding_mask is not None:
            padding_mask = tf.fill((batch_size, seqlen), -10000.0, dtype=scores.dtype)
            padding_mask = mask_fill(padding_mask, key_padding_mask, 0.0)
            scores = scores + tf.expand_dims(tf.expand_dims(padding_mask, axis=1), axis=1)
        # i am assuming this to be causal
        causal_mask = tf.experimental.numpy.triu(tf.fill((seqlen, seqlen), -10000.0), 1) # might need to be replaced
        scores = scores + tf.cast(causal_mask, dtype=scores.dtype)
        attention = tf.cast(tf.nn.softmax(scores, axis=-1), dtype=v.dtype)
        output = tf.einsum("bhts,bshd->bthd", attention, v)
        return output

class SelfAttention(nn.Module):
    def __init__(
        self,
        causal: bool = True,
        softmax_scale: Optional[float] = None,
        attention_dropout: float = 0.0,
    ) -> None:
        super().__init__()
        self.causal = causal
        self.softmax_scale = softmax_scale
        self.drop = nn.Dropout(attention_dropout)
    @torch.autocast("cpu", enabled=False)
    def forward(
        self,
        qkv: torch.FloatTensor,
        causal: bool = None,
        key_padding_mask: Optional[torch.BoolTensor] = None,
        **kwargs,
    ) -> torch.FloatTensor:
        batch_size, seqlen = qkv.shape[0], qkv.shape[1]
        q, k, v = qkv.unbind(dim=2)
        q = q.to(torch.float32)
        k = k.to(torch.float32)
        causal = self.causal if causal is None else causal
        softmax_scale = self.softmax_scale or 1.0 / math.sqrt(q.shape[-1])
        # Autocast is manually disabled to avoid `torch.einsum` performing the operation
        # using float16, which might lead to overflow
        scores = torch.einsum("bthd,bshd->bhts", q, k * softmax_scale)
        if key_padding_mask is not None:
            padding_mask = torch.full((batch_size, seqlen), -10000.0, dtype=scores.dtype, device=scores.device)
            padding_mask.masked_fill_(key_padding_mask, 0.0)
            scores = scores + rearrange(padding_mask, "b s -> b 1 1 s")
        if causal:
            causal_mask = torch.triu(torch.full((seqlen, seqlen), -10000.0, device=scores.device), 1)
            scores = scores + causal_mask.to(dtype=scores.dtype)
        attention = torch.softmax(scores, dim=-1).to(v.dtype)
        attention = self.drop(attention)
        output = torch.einsum("bhts,bshd->bthd", attention, v)
        return output

my_self_atten = MySelfAttention(softmax_scale=0.1)
self_atten = SelfAttention(softmax_scale=0.1)

last_dim = 5
# qkv = torch.randn([2,3,2, 8, last_dim*2])
# qkv = torch.randn([2,3, 8, last_dim*2]) # making qkv
qkv = torch.randn([7, 17, 3, 5, 11])
print(self_atten(qkv))
print("-----")
qkv = tf.constant(qkv.numpy())
my_self_atten(qkv)

tensor([[[[ 7.8216e-01,  1.6057e+00, -6.2888e-01,  ..., -1.1855e+00,
            2.1962e+00,  1.5852e-01],
          [ 6.8783e-02,  2.7639e+00, -5.0665e-01,  ..., -2.9511e-01,
            7.0020e-02,  2.5522e-01],
          [-1.3802e+00,  6.1853e-01,  1.2782e+00,  ..., -7.9474e-02,
            1.7819e+00,  9.7527e-02],
          [-6.9262e-01, -9.2186e-01, -1.0209e+00,  ..., -8.3433e-01,
            8.6667e-02,  1.5283e-01],
          [-1.1307e+00, -2.4051e-01,  4.5175e-01,  ..., -4.2752e-01,
           -1.2928e+00, -8.6309e-01]],

         [[ 7.7936e-02,  2.1183e-01,  3.2225e-02,  ..., -9.1052e-01,
            2.5773e-01, -1.0369e+00],
          [-1.1282e+00,  1.5443e+00,  2.6632e-01,  ...,  1.1326e-01,
            4.3200e-01, -5.7752e-03],
          [-1.3473e-01, -1.2908e-01,  3.9490e-01,  ..., -4.3477e-01,
            1.3972e+00, -4.1296e-01],
          [-8.6295e-01, -3.5388e-01, -8.1409e-01,  ..., -1.1982e+00,
            1.8707e-01, -1.1732e-01],
          [-4.5815e-01, -5.4771e-01

<tf.Tensor: shape=(7, 17, 5, 11), dtype=float32, numpy=
array([[[[ 7.82163501e-01,  1.60567415e+00, -6.28879726e-01, ...,
          -1.18546379e+00,  2.19618988e+00,  1.58521876e-01],
         [ 6.87834620e-02,  2.76394773e+00, -5.06650627e-01, ...,
          -2.95114070e-01,  7.00201616e-02,  2.55223244e-01],
         [-1.38016903e+00,  6.18527055e-01,  1.27824712e+00, ...,
          -7.94743299e-02,  1.78190470e+00,  9.75266472e-02],
         [-6.92622185e-01, -9.21856761e-01, -1.02088904e+00, ...,
          -8.34326744e-01,  8.66672173e-02,  1.52826503e-01],
         [-1.13069391e+00, -2.40513384e-01,  4.51752931e-01, ...,
          -4.27516937e-01, -1.29279351e+00, -8.63091290e-01]],

        [[ 7.79363215e-02,  2.11826324e-01,  3.22249830e-02, ...,
          -9.10517752e-01,  2.57727563e-01, -1.03686702e+00],
         [-1.12815976e+00,  1.54428232e+00,  2.66317725e-01, ...,
           1.13263369e-01,  4.31996763e-01, -5.77515364e-03],
         [-1.34734511e-01, -1.29076302e-01,  3

In [ ]:
class MyCrossAttention(tf.Module):
    def __init__(self, softmax_scale, name=None): # assume to be causal
        super().__init__(name)
        self.softmax_scale = softmax_scale
    def __call__(self, q, kv, key_padding_mask):
        batch_size, seqlen_q = q.shape[0], q.shape[1]
        seqlen_k = kv.shape[1]
        if kv.shape[3] != kv.shape[2]:
            kv = tf.repeat(kv, repeats=q.shape[2] // kv.shape[3], axis=-2)
        k, v = tf.unstack(kv, axis=2)
        q = tf.cast(q, dtype=tf.float32)
        k = tf.cast(k, dtype=tf.float32)
        scores = tf.einsum("bthd,bshd->bhts", q, k * self.softmax_scale)
        if key_padding_mask is not None:
            padding_mask = tf.fill((batch_size, seqlen_k), -10000.0, dtype=scores.dtype)
            padding_mask = mask_fill(padding_mask, key_padding_mask, 0.0)
            scores = scores + tf.expand_dims(tf.expand_dims(padding_mask, axis=1), axis=1)
        # causal stuff
        rows = tf.expand_dims(tf.range(seqlen_q, dtype=tf.long), axis=1) # if this fails, do axis=-1
        cols = tf.range(seqlen_k, dtype=tf.long)
        causal_mask = cols > rows + seqlen_k - seqlen_q
        scores = mask_fill(scores, causal_mask, -10000.0)
        # end of causal stuff
        attention = tf.softmax(scores, axis=-1, dtype=v.dtype)
        output = tf.einsum("bhts,bshd->bthd", attention, v)
        return output

In [ ]:
def MY_update_kv_cache(kv, inference_params, layer_idx:int):
    num_heads, head_dim = kv.shape[-2:]
    if layer_idx not in inference_params.key_value_memory_dict:
        # during the UDO desgin process, could use uninitalized memory rather than zeros
        inference_params.key_value_memory_dict[layer_idx] = tf.zeros(
            [
                inference_params.max_batch_size,
                inference_params.max_seqlen,
                2,
                num_heads,
                head_dim
            ],
            dtype=kv.dtype
        )
    batch_start = inference_params.batch_size_offset
    batch_end = batch_start + kv.shape[0]
    sequence_start = inference_params.seqlen_offset
    sequence_end = sequence_start + kv.shape[1]
    # When the current sequence length is equal to or larger than the maximum sequence length,
    # we need to concatenate the current `kv` with the cached `kv` to expand its length
    if sequence_end >= inference_params.max_seqlen:
        # the line below might fail due to the tuple
        inference_params.key_value_memory_dict[layer_idx] = tf.concat((inference_params.key_value_memory_dict[layer_idx], kv), axis=1)
    inference_params.key_value_memory_dict[layer_idx][batch_start:batch_end, sequence_start:sequence_end, ...] = kv
    kv = inference_params.key_value_memory_dict[layer_idx][batch_start:batch_end, :sequence_end, ...]
    return kv

In [1]:
class MyMHA(tf.Module):
    def __init__(self, dtype, rotary_dim, rotary_base, rotary_scale_base,
                 n_head, n_head_kv, head_dim, bias:bool, softmax_scale:float,
                 layer_idx:int, return_residual:bool,
                 flash_rot_emb:bool, n_positions, n_embd,
                 fused_dense:bool,
                 flash_attn:bool,
                 name=None):
        super().__init__(name)
        # Rotary embedding
        self.rotary_dim = rotary_dim
        if self.rotary_dim > 0:
            rotary_cls = FlashRotaryEmbedding if flash_rot_emb else MyRotaryEmbedding
            rotary_kwargs = {}
            if rotary_cls is RotaryEmbedding:
                rotary_kwargs["max_position_embeddings"] = n_positions
            self.rotary_emb =  rotary_cls(
                self.rotary_dim,
                base=rotary_base,
                scale_base=rotary_scale_base,
                kwargs= **rotary_kwargs,
            )
        # MLP
        self.n_head = n_head
        self.n_head_kv = n_head_kv
        self.head_dim = head_dim
        op_size = self.head_dim * (self.n_head + 2 * self.n_head_kv)
        hidden_size = n_embd
        linear_cls = FusedDense if fused_dense else bert.Dense
        self.Wqkv = linear_cls(hidden_size, op_size, weights=None,  
                               is_biased=True, bias=None)
        self.out_proj = linear_cls(hidden_size, hidden_size, weights=None,  
                               is_biased=True, bias=None)
        # Attention
        attn_cls = FlashSelfAttention if flash_attn else MySelfAttention
        cross_attn_cls = FlashCrossAttention if flash_attn else MyCrossAttention
        self.inner_attn = attn_cls(softmax_scale=softmax_scale)
        self.inner_cross_attn = cross_attn_cls(softmax_scale=softmax_scale)
        self.flash_attn = flash_attn and attn_cls is FlashSelfAttention
        self.layer_idx = layer_idx
        self.return_residual = return_residual
    
    def _forward_self_attn(self, x, key_padding_mask):
        qkv = self.Wqkv(x)
        h = qkv.shape[-1] // (3 * self.head_dim)
        assert(h * 3 * self.head_dim == qkv.shape[-1])
        qkv = tf.reshape(qkv, qkv.shape[:-1] + [3, h, self.head_dim])
        if self.rotary_dim > 0:
            qkv = self.rotary_emb(qkv)
        if self.flash_attn:
            batch_size, seqlen = qkv.shape[0], qkv.shape[1]
            cu_seqlens, max_seqlen = None, None
            if key_padding_mask is not None:
                # If `key_padding_mask` is supplied, we need to unpad the input and retrieve
                # the `cu_seqlens` and `max_seqlen` to be used by `flash-attn`
                qkv, indices, cu_seqlens, max_seqlen = unpad_input(qkv, key_padding_mask)
            attn_output = self.inner_attn(qkv, cu_seqlens=cu_seqlens, max_seqlen=max_seqlen)
            # If `key_padding_mask` is supplied, we need to pad the output back to the original shape
            return pad_input(attn_output, indices, batch_size, seqlen)
    
    def _forward_cross_attn(self, x, past_key_values, key_padding_mask):
        batch_size = x.shape[0]
        qkv = self.Wqkv(x)
        q = qkv[..., : self.n_head * self.head_dim]
        h = q.shape[-1] // self.head_dim
        assert(h * self.head_dim == q.shape[-1])
        q = tf.reshape(q, q.shape[:-1] + [h, self.head_dim])
        kv = qkv[..., self.n_head * self.head_dim :]
        hkv = kv.shape[-1] // (2 * self.head_dim)
        assert(hkv * 2 * self.head_dim == kv.shape[-1])
        kv = tf.reshape(kv, kv.shape[:-1] + [2, hkv, self.head_dim])
        seqlen_offset = past_key_values.seqlen_offset if past_key_values is not None else 0
        causal = None if seqlen_offset == 0 else False
        if self.rotary_dim > 0:
            q, kv = self.rotary_emb(q, kv=kv, seqlen_offset=seqlen_offset)
        if past_key_values is not None:
            kv = MY_update_kv_cache(kv, past_key_values, self.layer_idx)
        assert(self.flash_attn is not None)
        batch_size, seqlen_q = q.shape[0], q.shape[1]
        seqlen_k = kv.shape[1]
        cu_seqlens_q, cu_seqlens_k, max_seqlen_q, max_seqlen_k = (
            None,
            None,
            None,
            None,
        )
        if key_padding_mask is not None:
            kv, _, cu_seqlens_k, max_seqlen_k = unpad_input(kv, key_padding_mask)
            if seqlen_q == 1:
                key_padding_mask = tf.ones((batch_size, 1))
            elif seqlen_q != seqlen_k:
                key_padding_mask = key_padding_mask[:, -seqlen_q:]
            q, indices_q, cu_seqlens_q, max_seqlen_q = unpad_input(q, key_padding_mask)
        attn_output = self.inner_cross_attn(
            q,
            kv,
            causal=causal,
            cu_seqlens=cu_seqlens_q,
            max_seqlen=max_seqlen_q,
            cu_seqlens_k=cu_seqlens_k,
            max_seqlen_k=max_seqlen_k,
        )
        return (
        pad_input(attn_output, indices_q, batch_size, max_seqlen_q)
        if key_padding_mask is not None
        else attn_output
        )
        
    def __call__(self, x, past_key_values, attention_mask):
        assert(attention_mask is not None)
        attention_mask = tf.cast(attention_mask, dtype=tf.bool)
        # MHA
        if self.n_head == self.n_head_kv:
            assert(past_key_values is None)
            # If `past_key_values` are not supplied, we run self-attention
            attn_output = self._forward_self_attn(x, attention_mask)
        # MQA / GQA
        else:
            # Regardless of `past_key_values` being supplied or not, it always use cross-attention
            # because `q` and `kv` lengths might be different
            attn_output = self._forward_cross_attn(x, past_key_values, attention_mask
        output = tf.reshape(attn_output, attn.shape[:-2] + [attn_output.shape[-1] * attn_output.shape[-2]])
        output = self.out_proj(output)
        return output if not self.return_residual else (output, x)

SyntaxError: invalid syntax (2711594939.py, line 21)

In [ ]:
class MyParallelBlock(tf.Module):
    def __init__(self, block_idx, name=None):
        super().__init__(name)
        self.ln = bert.LayerNorm(weights=None, biases=None, eps=None)
        self.block_idx = block_idx
        self.mixer = MyMHA()
        self.mlp = MyMLP()
    
    def __call__(self, hidden_states, past_key_values, attention_mask):
        residual = hidden_states
        hidden_states = self.ln(hidden_states)
        attn_outputs = self.mixer(
            hidden_states,
            past_key_values=past_key_values,
            attention_mask=attention_mask,
        )
        if isinstance(attn_outputs, tuple):
            attn_outputs = attn_outputs[0]
        feed_forward_hidden_states = self.mlp(hidden_states)
        hidden_states = attn_outputs + feed_forward_hidden_states + residual
        return hidden_states

In [ ]:
class MyCausalLMHead(tf.Module):
    def __init__(self, n_embd, layer_norm_eps, vocab_size, name=None):
        super().__init__(name)
        self.ln = nn.LayerNorm(n_embd, eps=layer_norm_eps)
        self.linear = bert.Dense(n_embd, vocab_size)
    def __call__(self, hidden_states):
        hidden_states = self.ln(hidden_states)
        logits = tf.cast(self.linear(hidden_states), dtype=tf.float32)
        return logits

In [ ]:
# class MyPhiPreTrainedModel(tf.Module):
    